In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from super_resolution_task import *
from bayescap import *

## Loading the pre-trained (frozen) SRGAN base model and instantiating BayesCap

In [2]:
device = "cuda:0"
NetGen = SRGenerator()
NetGen.load_state_dict(torch.load("../ckpt/srgan-ImageNet-bc347d67.pth", map_location=device))
# 
model_parameters = filter(lambda p: True, NetGen.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print("Number of Parameters:", params)
# 
NetCap = SRBayesCap(in_channels=3, out_channels=3)
NetCap.load_state_dict(torch.load('../ckpt/BayesCap_SRGAN_best.pth', map_location=device))

Number of Parameters: 1547350


## Evaluating BayesCap

### Set5 Dataset

In [ ]:
dataset_test = SRDataset(data_root="../data/Set5/original", image_size=(256,256), upscale_factor=4)
loader_test = DataLoader(dataset_test, batch_size=1, pin_memory=True, shuffle=False)
eval_BayesCap(NetCap, NetGen, loader_test, device=device, dtype=torch.cuda.FloatTensor, viz=True, test=True)

### Set14 Dataset

In [ ]:
dataset_test = SRDataset(data_root="../data/Set14/original", image_size=(256, 256), upscale_factor=4)
loader_test = DataLoader(dataset_test, batch_size=1, pin_memory=True, shuffle=False)
eval_BayesCap(NetCap, NetGen, loader_test, device=device, dtype=torch.cuda.FloatTensor, viz=False, test=True)

### BSD100 Dataset

In [ ]:
dataset_test = SRDataset(data_root="../data/BSDS100/original", image_size=(256, 256), upscale_factor=4)
loader_test = DataLoader(dataset_test, batch_size=1, pin_memory=True, shuffle=False)
eval_BayesCap(NetCap, NetGen, loader_test, device=device, dtype=torch.cuda.FloatTensor, viz=False, test=True)